In [1]:
from pybedtools import BedTool
from collections import defaultdict
import csv
import pandas as pd
import numpy as np

In [2]:
def geneLoc(gtf):
    exons=BedTool(gtf)
    genePos=defaultdict(list)
    geneChr={}
    geneStr={}
    for lg in exons:
        gn=lg.attrs['gene_id']
        genePos[gn].append(int(lg.start))
        genePos[gn].append(int(lg.end))
        geneStr[gn]=str(lg.strand)
        geneChr[gn]=str(lg.chrom)
    geneLoc={}
    for gene in genePos:
        geneLoc[gene]=(geneChr[gene],min(genePos[gene]),max(genePos[gene]),geneStr[gene])
    return geneLoc

In [3]:
genPos={}

In [ ]:
genPos['Pliv']=geneLoc(open("gtfs/Pliv_aH2p.gtf"))

In [ ]:
genPos['Bflo']=geneLoc(open("gtfs/amphioxus_7u5tJ.gtf"))

In [5]:
synt=[]
for r in csv.reader(open('Hsa_Mmu_synt.txt'),delimiter='\t'):
    if r[0]=='s1g': continue
    rr=[int(e) if j in (1,3,5,7) else e for j,e in enumerate(r)]
    synt.append(rr)

In [4]:
def readSynt(sfn):
    synt=[]
    for r in csv.reader(open(sfn),delimiter='\t'):
        if r[0]=='s1g': continue
        rr=[int(e) if j in (1,3,5,7) else e for j,e in enumerate(r)]
        synt.append(rr)
    syntSrt=sorted(synt,key=lambda x:(x[2],x[3],x[6],x[7]))
    return(syntSrt)

In [5]:
def syntBlks(syntSrt):
    blocks=[]
    offset=2
    blk=[]
    i=0
    while i<len(syntSrt):
        r=syntSrt[i]
        #print('>',i,r)
        for j in range(i+1,len(syntSrt),1):
            nr=syntSrt[j]
            #print(j,nr)
            if r[2]==nr[2] and r[6]==nr[6]:
                neg=[r[7]-k for k in range(1,1+offset,1)]
                pos=[r[7]+k for k in range(1,1+offset,1)]
                #print(neg,pos)
                if nr[7] in neg or nr[7] in pos:
                    #print('+',i+1+j,nr)
                    blk.append(nr)
                    #print(' ',i+1+j,nr)
                    r=nr
                    i=j
                    continue
            else:
                break
        if len(blk)>1:
            blocks.append(blk)
        #print(len(blk))
        blk=[]
        i+=1
    return(blocks)


In [54]:
pliv_spur_synt=readSynt('Pliv_Spur_synt.txt')

In [55]:
pliv_lvar_synt=readSynt('Pliv_Lvar_synt.txt')

In [59]:
spur_lvar_synt=readSynt('Spur_Lvar_synt.txt')

In [56]:
hsap_mmus_synt=readSynt('Hsa_Mmu_synt.txt')

In [57]:
hsap_ggal_synt=readSynt('Homsap-Galgal_reci_synt.tsv')

In [60]:
hsap_locu_synt=readSynt('Homsap-Lepocu_reci_synt.tsv')

In [84]:
arub_pliv_synt=readSynt('Arub_Pliv_synt.txt')

In [91]:
arub_pmin_synt=readSynt('Arub_Pmin_synt.txt') 

In [61]:
pliv_spur_blk=syntBlks(pliv_spur_synt)

In [62]:
pliv_lvar_blk=syntBlks(pliv_lvar_synt)

In [63]:
spur_lvar_blk=syntBlks(spur_lvar_synt)

In [69]:
hsap_mmus_blk=syntBlks(hsap_mmus_synt)

In [65]:
hsap_ggal_blk=syntBlks(hsap_ggal_synt)

In [92]:
arub_pmin_blk=syntBlks(arub_pmin_synt)

In [87]:
arub_pliv_blk=syntBlks(arub_pliv_synt)

In [88]:
checkBlk('Arub','Pliv',arub_pliv_blk,arub_pliv_synt)

{'sp1': 'Arub',
 'sp2': 'Pliv',
 'nb_blocks': 297,
 'nb_genes': 9035,
 'med_block_size': 2.0,
 'genes_in_block': 692,
 'frac_in_block': 7.66}

In [42]:
def checkBlk(sp1,sp2,blk,synt):
    blsz=[len(bl) for i,bl in enumerate(blk)]
    out={'sp1':sp1,'sp2':sp2,'nb_blocks':len(blsz),'nb_genes':len(synt),'med_block_size':np.median(blsz),'genes_in_block':sum(blsz),'frac_in_block':round(sum(blsz)/len(synt)*100,2)}
    return(out)

In [33]:
import pandas as pd

In [74]:
df = pd.DataFrame(columns=["sp1","sp2","nb_blocks", "nb_genes","med_block_size","genes_in_block","frac_in_block"])

In [75]:
df=df.append(checkBlk('Pliv','Lvar',pliv_lvar_blk,pliv_lvar_synt),ignore_index=True)

In [76]:
df=df.append(checkBlk('Pliv','Spur',pliv_spur_blk,pliv_spur_synt),ignore_index=True)

In [77]:
df=df.append(checkBlk('Spur','Lvar',spur_lvar_blk,spur_lvar_synt),ignore_index=True)

In [78]:
df=df.append(checkBlk('Hsap','Mmus',hsap_mmus_blk,hsap_mmus_synt),ignore_index=True)

In [79]:
df=df.append(checkBlk('Hsap','Ggal',hsap_ggal_blk,hsap_ggal_synt),ignore_index=True)

In [80]:
df=df.append(checkBlk('Hsap','Locu',hsap_locu_blk,hsap_locu_synt),ignore_index=True)

In [89]:
df=df.append(checkBlk('Arub','Pliv',arub_pliv_blk,arub_pliv_synt),ignore_index=True)

In [93]:
df=df.append(checkBlk('Arub','Pmin',arub_pmin_blk,arub_pmin_synt),ignore_index=True)

In [94]:
df.to_csv('test.Txt',sep='\t')

In [45]:
hsap_mmus_blsz=[len(bl) for i,bl in enumerate(hsap_mmus_blk)]

NameError: name 'hsap_mmus_blk' is not defined

In [58]:
pliv_spur_blsz=[len(bl) for i,bl in enumerate(spur_pliv_blk)]

In [59]:
print(len(pliv_spur_blsz),np.median(pliv_spur_blsz),sum(pliv_spur_blsz),round(sum(pliv_spur_blsz)/len(pliv_spur_synt)*100,2))

1035 5.0 7391 62.75


In [60]:
print(len(hsap_mmus_blsz),np.median(hsap_mmus_blsz),sum(hsap_mmus_blsz),round(sum(hsap_mmus_blsz)/len(hsap_mmus_synt)*100,2))

433 19.0 13963 88.18


In [31]:
list(range(1,1+2,1))

[1, 2]

In [34]:
spur_pliv_blk=syntBlks(synt)

In [22]:
blocks=[]
offset=2
blk=[]
i=0
while i<len(syntSrt):
    r=syntSrt[i]
    #print('>',i,r)
    for j in range(i+1,len(syntSrt),1):
        nr=syntSrt[j]
        #print(j,nr)
        if r[2]==nr[2] and r[6]==nr[6]:
            neg=[r[7]-k for k in range(1,1+offset,1)]
            pos=[r[7]+k for k in range(1,1+offset,1)]
            #print(neg,pos)
            if nr[7] in neg or nr[7] in pos:
                #print('+',i+1+j,nr)
                blk.append(nr)
                #print(' ',i+1+j,nr)
                r=nr
                i=j
                continue
        else:
            break
    if len(blk)>1:
        blocks.append(blk)
    #print(len(blk))
    blk=[]
    i+=1
            
        
        
    
    #if i==10: break

In [18]:
print(len(syntSrt))

15835


In [35]:
print(len(syntSrt))

11778


In [48]:
ps_blsz=[len(bl) for i,bl in enumerate(spur_pliv_blk)]

In [49]:
print(ps_blsz,sum(ps_blsz),round(sum(ps_blsz)/len(syntSrt)*100,2))

[4, 8, 8, 6, 7, 9, 3, 8, 6, 4, 13, 4, 2, 22, 16, 3, 6, 16, 2, 4, 2, 8, 7, 3, 2, 14, 4, 2, 6, 16, 2, 5, 7, 8, 2, 6, 2, 4, 4, 6, 2, 3, 4, 4, 4, 2, 6, 2, 2, 2, 19, 3, 5, 8, 3, 10, 7, 2, 2, 2, 3, 12, 2, 5, 8, 2, 3, 22, 28, 2, 5, 3, 3, 9, 12, 21, 4, 10, 6, 33, 6, 4, 21, 27, 8, 5, 8, 18, 10, 19, 7, 17, 6, 2, 3, 4, 5, 2, 3, 17, 2, 3, 3, 5, 4, 21, 3, 8, 2, 3, 23, 2, 12, 17, 9, 29, 4, 4, 8, 4, 28, 7, 12, 5, 5, 5, 8, 2, 11, 24, 16, 4, 5, 10, 3, 5, 12, 3, 10, 4, 7, 7, 2, 7, 31, 11, 8, 2, 9, 4, 5, 2, 3, 2, 12, 9, 9, 14, 2, 15, 21, 69, 3, 8, 7, 4, 6, 11, 7, 5, 5, 11, 11, 3, 5, 6, 2, 2, 17, 2, 2, 2, 2, 3, 3, 6, 24, 2, 2, 3, 2, 3, 2, 5, 8, 8, 9, 8, 2, 4, 54, 7, 7, 10, 15, 13, 10, 5, 5, 6, 3, 2, 5, 46, 5, 7, 8, 2, 16, 2, 4, 6, 11, 12, 4, 3, 2, 4, 2, 4, 3, 2, 3, 5, 10, 18, 24, 12, 5, 3, 3, 10, 48, 19, 10, 6, 3, 18, 4, 5, 9, 8, 6, 19, 3, 2, 2, 4, 4, 4, 9, 7, 2, 11, 8, 4, 2, 2, 2, 3, 4, 2, 4, 2, 2, 4, 4, 3, 2, 11, 3, 10, 8, 3, 3, 9, 7, 8, 3, 3, 4, 21, 7, 5, 2, 4, 3, 3, 2, 4, 2, 3, 9, 5, 7, 3, 3, 15, 8, 3

In [23]:
print(len(blocks))

433


In [25]:
blsz=[len(bl) for i,bl in enumerate(blocks)]

In [28]:
print(blsz,sum(blsz),sum(blsz)/len(syntSrt))

[91, 187, 153, 37, 2, 46, 86, 29, 3, 68, 21, 78, 13, 58, 16, 78, 70, 37, 5, 10, 11, 302, 67, 14, 16, 33, 13, 5, 12, 2, 3, 18, 32, 7, 23, 15, 4, 3, 4, 3, 15, 2, 3, 15, 6, 82, 34, 23, 220, 76, 18, 38, 24, 103, 3, 33, 108, 40, 148, 61, 35, 14, 100, 63, 140, 22, 17, 22, 12, 29, 37, 8, 2, 6, 3, 65, 31, 5, 3, 49, 54, 61, 4, 87, 61, 84, 26, 25, 42, 10, 31, 66, 15, 17, 10, 20, 16, 22, 42, 4, 11, 6, 17, 45, 6, 39, 4, 45, 47, 3, 26, 55, 20, 2, 22, 8, 65, 35, 192, 3, 6, 3, 10, 114, 18, 3, 31, 4, 135, 8, 16, 6, 34, 25, 2, 5, 97, 14, 70, 5, 27, 32, 2, 46, 129, 39, 18, 78, 15, 9, 46, 22, 19, 68, 30, 12, 46, 3, 4, 4, 15, 12, 37, 59, 48, 23, 12, 3, 8, 188, 3, 16, 7, 8, 10, 18, 29, 82, 32, 20, 27, 10, 8, 53, 13, 38, 5, 10, 15, 7, 2, 14, 42, 4, 71, 15, 2, 65, 10, 109, 106, 4, 40, 9, 2, 5, 4, 9, 24, 58, 19, 18, 6, 94, 4, 48, 19, 3, 6, 18, 38, 32, 5, 9, 10, 50, 2, 6, 17, 6, 24, 13, 7, 16, 2, 87, 54, 7, 6, 29, 216, 21, 23, 127, 11, 36, 194, 59, 6, 20, 18, 11, 3, 13, 22, 11, 2, 9, 9, 19, 46, 3, 4, 6, 187, 6